In [11]:
%load_ext autoreload
%autoreload 2


from util2 import *
from myalgorithm_0629_1 import algorithm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- 실험해볼 내용들

1. 출발지 내에서의 가중치, 도착지 내에서의 가중치에 이어 주문 완료 시간과 주문 데드라인 시간, 출발지와 도착지 사이의 거리 또한 가중치를 주면 어떨까?

## 초기에 비슷한 번들로 먼저 묶어서 테스트 해보기

#### Weight 최적화

In [ ]:
## Basic Code
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

## Custom Code
def find(v):
    while v != parent[v]:
        parent[v] = parent[parent[v]]
        v = parent[v]

    return v

def union(a, b, new_bundle):
    if a > b:
        a, b = b, a

    parent[b] = a
    all_bundles[a] = new_bundle

for weight1 in range(-30, 30):
    weight1 /= 10
    for weight2 in range(-30, 30):
        weight2 /= 10

        weight1 = round(weight1, 1)
        weight2 = round(weight2, 1)

        print(weight1, weight2)

        car_rider = ALL_RIDERS[2]
        all_bundles_original = []

        for ord in ALL_ORDERS:
            new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
            car_rider.available_number -= 1
            all_bundles_original.append(new_bundle)

        all_bundles = all_bundles_original.copy()

        for i in range(len(all_bundles)):
            bundle = all_bundles[i]

            shop_seq = bundle.shop_seq

            xs_s_sum = 0
            ys_s_sum = 0

            xs_e_sum = 0
            ys_e_sum = 0

            readytimes_sum = 0
            deadlines_sum = 0

            shop_seq_len = len(shop_seq)

            for order_num in shop_seq:
                order = ALL_ORDERS[order_num]

                xs_s_sum += order.shop_lat
                ys_s_sum += order.shop_lon

                xs_e_sum += order.dlv_lat
                ys_e_sum += order.dlv_lon

                readytimes_sum += order.ready_time
                deadlines_sum += order.deadline

            xs_s_avg = xs_s_sum / shop_seq_len
            ys_s_avg = ys_s_sum / shop_seq_len

            xs_e_avg = xs_e_sum / shop_seq_len
            ys_e_avg = ys_e_sum / shop_seq_len

            readytimes_avg = readytimes_sum / shop_seq_len
            deadlines_avg = deadlines_sum / shop_seq_len

            avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

            bundle.avg_info = avg_info

        edges = []
        for i in range(len(all_bundles)):
            for j in range(i + 1, len(all_bundles)):
                avg_info1 = all_bundles[i].avg_info
                avg_info2 = all_bundles[j].avg_info

                sx1, sy1, ex1, ey1, r1, d1 = avg_info1
                sx2, sy2, ex2, ey2, r2, d2 = avg_info2

                caluled_dist1 = get_dist_by_coords(sx1, sy1, sx2, sy2)
                caluled_dist2 = get_dist_by_coords(ex1, ey1, ex2, ey2)

                r_diff = abs(r1 - r2)
                d_diff = abs(d1 - d2)

                start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

                # weight1 = 0
                # weight2 = 0

                dist1 = DIST[i][j]
                dist2 = DIST[i + K][j + K]

                diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

                edges.append((i, j, diff_score))

        parent = list(range(K))
        edges.sort(key=lambda x: x[2])

        for bundle_num1, bundle_num2, diff_score in edges:
            rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

            if rbn1 == rbn2:
                continue
            
            new_bundle = try_merging_bundles4(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2])
            if new_bundle is not None:
                all_bundles[rbn1].rider.available_number += 1
                all_bundles[rbn2].rider.available_number += 1
                new_bundle.rider.available_number -= 1

                union(rbn1, rbn2, new_bundle)

        parent = [find(v) for v in parent]

        result_bundles = [all_bundles[v] for v in set(parent)]

        solution = [
            # rider type, shop_seq, dlv_seq
            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq] for bundle in result_bundles
        ]

        ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]
        for r in ALL_RIDERS:
            r.T = np.round(DIST/r.speed + r.service_time).astype(int)

        checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

        print(weight1, weight2, sum((bundle.cost for bundle in result_bundles)) / K)

> 결과는 alg_test_0629_1_weight_search에 저장함

In [ ]:
1.0 -1.7
1.0 -1.7 3672.6479999999997
1.0 -1.6
1.0 -1.6 3672.6479999999997
1.0 -1.5
1.0 -1.5 3672.6479999999997
1.0 -1.4
1.0 -1.4 3672.6479999999997

### Weight 이용

In [27]:
len(DIST)

400

In [37]:
len(ALL_ORDERS)

200

In [51]:
## Basic Code
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

## Custom Code
def find(v):
    while v != parent[v]:
        parent[v] = parent[parent[v]]
        v = parent[v]

    return v

def union(a, b, new_bundle):
    if a > b:
        a, b = b, a

    parent[b] = a
    all_bundles[a] = new_bundle

for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        K = prob['K']

        ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
        ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

        DIST = np.array(prob['DIST'])
        for r in ALL_RIDERS:
            r.T = np.round(DIST/r.speed + r.service_time).astype(int)

        car_rider = ALL_RIDERS[2]

        all_bundles = []
        for ord in ALL_ORDERS:
            new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
            car_rider.available_number -= 1
            all_bundles.append(new_bundle)

        for i in range(len(all_bundles)):
            bundle = all_bundles[i]

            shop_seq = bundle.shop_seq

            xs_s_sum = 0
            ys_s_sum = 0

            xs_e_sum = 0
            ys_e_sum = 0

            readytimes_sum = 0
            deadlines_sum = 0

            shop_seq_len = len(shop_seq)

            for order_num in shop_seq:
                order = ALL_ORDERS[order_num]

                xs_s_sum += order.shop_lat
                ys_s_sum += order.shop_lon

                xs_e_sum += order.dlv_lat
                ys_e_sum += order.dlv_lon

                readytimes_sum += order.ready_time
                deadlines_sum += order.deadline

            xs_s_avg = xs_s_sum / shop_seq_len
            ys_s_avg = ys_s_sum / shop_seq_len

            xs_e_avg = xs_e_sum / shop_seq_len
            ys_e_avg = ys_e_sum / shop_seq_len

            readytimes_avg = readytimes_sum / shop_seq_len
            deadlines_avg = deadlines_sum / shop_seq_len

            avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

            bundle.avg_info = avg_info

        edges = []
        for i in range(len(all_bundles)):
            for j in range(i + 1, len(all_bundles)):
                avg_info1 = all_bundles[i].avg_info
                avg_info2 = all_bundles[j].avg_info

                sx1, sy1, ex1, ey1, r1, d1 = avg_info1
                sx2, sy2, ex2, ey2, r2, d2 = avg_info2

                caluled_dist1 = get_dist_by_coords(sx1, sy1, sx2, sy2)
                caluled_dist2 = get_dist_by_coords(ex1, ey1, ex2, ey2)

                r_diff = abs(r1 - r2)
                d_diff = abs(d1 - d2)

                start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

                # weight1 = 1
                # weight2 = -1.5

                # weight1 = 0
                # weight2 = -1.5

                dist1 = DIST[i][j]
                dist2 = DIST[i + K][j + K]

                diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

                edges.append((i, j, diff_score))

        parent = list(range(K))
        edges.sort(key=lambda x: x[2])

        for bundle_num1, bundle_num2, diff_score in edges:
            rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

            if rbn1 == rbn2:
                continue
            
            new_bundle = try_merging_bundles4(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2])
            if new_bundle is not None:
                all_bundles[rbn1].rider.available_number += 1
                all_bundles[rbn2].rider.available_number += 1
                new_bundle.rider.available_number -= 1

                union(rbn1, rbn2, new_bundle)

        parent = [find(v) for v in parent]

        result_bundles = [all_bundles[v] for v in set(parent)]

        # solution = [
        #     # rider type, shop_seq, dlv_seq
        #     [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq] for bundle in result_bundles
        # ]

        # ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]
        # for r in ALL_RIDERS:
        #     r.T = np.round(DIST/r.speed + r.service_time).astype(int)

        # checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

        print(weight1, weight2, sum((bundle.cost for bundle in result_bundles)) / K)

0 -3 3887.276
0 -2 3895.8479999999995
0 -1.5 3845.8639999999996
0 -1 3899.5299999999993
0 -0.5 3912.446
0 0 3977.244
0 0.5 3870.0180000000005
1 -3 3800.5499999999993
1 -2 3749.6819999999993
1 -1.5 3672.6479999999997
1 -1 3772.486
1 -0.5 3864.4339999999997
1 0 3874.47
1 0.5 3883.886


> 기존에는 출발지 사이의 거리와 도착지 사이의 거리만 이용하였으며, 추가적으로 주문 완료 시간과 주문 데드라인 시간, 출발지와 도착지 사이의 거리를 각각 가중치를 주어봤다. 그 결과 확실히 기존에 비해 총 비용이 줄어드는 경우가 있으나, 그렇게 크게 줄어든다는 생각은 안들며, 테스트케이스마다 최적의 가중치 조합이 다르다. 그러므로 최적의 가중치 조합을 시간 부담이 되지 않는 선에서 돌려보는 것이 괜찮을 것 같다.